# 07. Logistic Regression

- 분류 문제를 불 수 있는 가장 간단한 방법론
- 분류 문제를 다루는 수업에서 가장 처음으로 나오는 내용
- Cross Entropy Loss function이 무엇인지 알면 더 좋음
- Cost function이 줄어든다고 해서 좋은 것은 아니다!
- 풀고자하는 문제에 따라서 적절하게 cost function을 디자인하고 이용해 최적화!!!!
- 일반적으로 많이 사용하는 Cross Entropy!

## CE (Cross Entropy)
- Information Theory에서 나옴
- 어떠한 probability distribution이 2개 있을 때(p, q) 이 확률 분포 사이의 거리에 해당하는 의미를 가짐

### CE in neural network
<img src='../images/logistic001.png' height=300>
- 정답에 해당하는 확률 분포
- 뉴럴넷이 뱉어내는 확률 분포
- 현재 분류하고 있는 물건은 5개! 5개의 확률 분포는 모두 0보다 크고 합치면 1
- 뉴럴넷의 아웃풋의 합은 1이되기 어려워서 각각의 값에 exponential을 취해줌
- 각각의 값들이 모두 0보다 커지고, 더한 것으로 나눠주면 다 더할 경우 1이 됨 (softmax)
- 정답과 비교해서 거리를 줄임!!! 
- 클래스에 해당하는 확률은 1 나머지는 0인 one hot 코딩되어 있는 target value와 뉴럴넷의 결과로 나온 것을 softmax해서 확률로 바꾸고! 그 확률 사이의 거리를 CE로 정의하고 그것을 줄이는 방향으로 뉴럴넷을 학습시킨다!
- 클래스가 100개고 3번째 클래스에 속하면, 3번째 클래스만 신경쓰고 나머지 99개는 신경쓰지 않겠다!!! 
- 정답에 해당하는 index에 해당하는 숫자만 높이고 나머지는 신경쓰지 않겠다



<img src='../images/logistic002.png' height=300>

- computed : softmax를 취해서 나온 값(확률 분포)
- squared loss : 뉴럴넷의 결과와 내 결과를 빼서 제곱. 정답이 아닌 인덱스는 낮추고, 정답인 인덱스를 높이려고 함
- 반면 cross entropy는 정답인 인덱스만 초점을 맞춤
- cross entropy가 squared loss보다 분류에선 더 잘됨


In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
print ("PACKAGES LOADED")
print ("TF VERSION IS {version}".format(version=tf.__version__))

PACKAGES LOADED
TF VERSION IS 1.3.0


In [3]:
mnist      = input_data.read_data_sets('data/', one_hot=True)
trainimg   = mnist.train.images
trainlabel = mnist.train.labels
testimg    = mnist.test.images
testlabel  = mnist.test.labels
print ("MNIST loaded")

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
MNIST loaded


In [11]:
x = tf.placeholder("float", [None, 784])
y = tf.placeholder("float", [None, 10])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

In [13]:
actv = tf.nn.softmax(tf.matmul(x, W) + b)
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(actv), reduction_indices=1)) 

In [14]:
learning_rate = 0.01
optm = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

- pred = tf.equal() 뉴럴넷이 뱉은 라벨과 같으면 1, 아니면 0
- 평균을 내주면 accuracy가 나옴

In [17]:
pred = tf.equal(tf.argmax(actv, 1), tf.argmax(y, 1))
accr = tf.reduce_mean(tf.cast(pred,"float"))
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [18]:
training_epochs = 50
batch_size      = 100
display_step    = 5
# SESSION 열고
sess = tf.Session()
sess.run(init)
# 미니배치로 학습
for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = int(mnist.train.num_examples/batch_size) 
    # 55000/100 = 550 iteration
    for i in range(num_batch): 
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys})
        
        feeds = {x: batch_xs, y: batch_ys}
        avg_cost += sess.run(cost, feed_dict=feeds)/num_batch # 성능
        
    # DISPLAY
    if epoch % display_step == 0:
        feeds_train = {x: batch_xs, y: batch_ys}
        feeds_test = {x: mnist.test.images, y: mnist.test.labels}
        train_acc = sess.run(accr, feed_dict=feeds_train)
        test_acc = sess.run(accr, feed_dict=feeds_test)
        print ("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f" 
               % (epoch, training_epochs, avg_cost, train_acc, test_acc))
print ("DONE")

Epoch: 000/050 cost: 1.176892365 train_acc: 0.800 test_acc: 0.852
Epoch: 005/050 cost: 0.441032667 train_acc: 0.880 test_acc: 0.895
Epoch: 010/050 cost: 0.383345203 train_acc: 0.910 test_acc: 0.904
Epoch: 015/050 cost: 0.357281481 train_acc: 0.890 test_acc: 0.909
Epoch: 020/050 cost: 0.341470933 train_acc: 0.910 test_acc: 0.912
Epoch: 025/050 cost: 0.330546705 train_acc: 0.890 test_acc: 0.914
Epoch: 030/050 cost: 0.322359660 train_acc: 0.940 test_acc: 0.915
Epoch: 035/050 cost: 0.315943988 train_acc: 0.960 test_acc: 0.917
Epoch: 040/050 cost: 0.310740261 train_acc: 0.920 test_acc: 0.918
Epoch: 045/050 cost: 0.306383077 train_acc: 0.970 test_acc: 0.919
DONE
